# Parsing IFC

### Loading IFC data  
To explore the IFC model, we use the [IfcOpenShell](https://docs.ifcopenshell.org/ifcopenshell-python/installation.html) module, you will have to install it again when using a virtual environment. <br>


In [66]:
import ifcopenshell as ios

model = ios.open("../datasets/heartbreak_hotel.ifc")

### Iterating over the IfcStair instances 
We want to iterate over all stair instances and their properties and drop them into a list.<br> 
The same applies to relational object instances, as these define other objects that may be related to a stair flight, such as railings... 

In [67]:
ifcstairs = model.by_type('IfcStair')
relations = model.by_type('IfcRelAggregates')
print(str(len(ifcstairs))+' IfcStairs found')
ifcdoors = model.by_type('IfcDoor')
print(str(len(ifcdoors))+' Ifcdoors found')

3 IfcStairs found
18 Ifcdoors found


### Drilling down the IfcModel in search for the IfcStair properties
For convenience, we use a custom Python class that represents a stair concept. 

In [68]:
class Stair:
    def __init__(self, name, riserheight, treadlength, railings, numberofriser):
        self.name = name
        self.RiserHeight = riserheight
        self.TreadLength = treadlength
        self.NumberOfRailings = railings
        self.NumberOfRiser = numberofriser

class Door:
    def __init__(self, name, width, height):
        self.name = name
        self.width = width
        self.height = height


In [69]:
stair_list = []

for ifcstair in ifcstairs:
    if hasattr(ifcstair, "IsDefinedBy"):
        for rel in ifcstair.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties"):
                property_set = rel.RelatingPropertyDefinition
                if property_set.is_a("IfcPropertySet") and property_set.Name == "Pset_StairCommon":
                    if hasattr(property_set, "HasProperties"):
                        for property in property_set.HasProperties:
                            if property.is_a("IfcPropertySingleValue"):
                                if property.Name == "RiserHeight":
                                    riserheight = round(property.NominalValue.wrappedValue)
                                if property.Name == "TreadLength":
                                    treadlength = round(property.NominalValue.wrappedValue)
                                if property.Name == "NumberOfRiser":
                                    numberofriser = int(property.NominalValue.wrappedValue)
    numberofrailings=0
    for rel in relations:
        for related_element in rel.RelatedObjects:
            if related_element.is_a("IfcRailing"):
                if rel.RelatingObject.GlobalId == ifcstair.GlobalId:
                    numberofrailings +=1
    
    new_stair = Stair(ifcstair.Name, riserheight, treadlength, numberofrailings, numberofriser)
    stair_list.append(new_stair)
    
print(len(stair_list), "stairs created.")

3 stairs created.


In [70]:
door_list = []

for ifcdoor in ifcdoors:
    # name = ""
    # width = 0.0
    if hasattr(ifcdoor, "Name"):
        name = ifcdoor.Name
    if hasattr(ifcdoor, "OverallWidth"):
        width = ifcdoor.OverallWidth
    if hasattr(ifcdoor, "OverallHeight"):
        height = ifcdoor.OverallHeight
    
    new_door = Door(name=name, width=width, height=height)
    door_list.append(new_door)

print(len(door_list), "doors created.")

18 doors created.


### Dumping the stair & door data
Finally, we write the list of stair & door objects to a json file. In an intermediate step, we transform the list into a dictionary. Note that the json module has to be imported first. 

In [71]:
import json

stairs_dict = [stair.__dict__ for stair in stair_list]
with open("stairs.json", "w") as json_file:
    json.dump(stairs_dict, json_file, indent=4)

doors_dict = [door.__dict__ for door in door_list]
with open("doors.json", "w") as json_file:
    json.dump(doors_dict, json_file, indent=4)

## Convert our extracted data info an RDF graph

In [72]:
# Import needed components from rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, RDFS, OWL, URIRef

# initiate triple store, i.e. Graph()
g = Graph()

# Add namespaces and prefixes for ontologies
BOT = Namespace("https://w3id.org/bot#")
g.bind("bot", BOT)
BEO = Namespace("https://w3id.org/beo#")
g.bind("beo", BEO)
# Add namespace and prefix for instance graph (ABox)
INST = Namespace("https://example.org#")
g.bind("ex", INST) # bind to default empty prefix

# Our instances

for stair in stair_list:
    s = INST[str(stair.name).replace(" ","").replace(":","_")]
    g.add((s, RDF.type, BEO.Stair))
    g.add((s, INST["RiserHeight"], Literal(stair.RiserHeight)))
    g.add((s, INST["TreadLength"], Literal(stair.TreadLength)))
    g.add((s, INST["NumberOfRailings"], Literal(stair.NumberOfRailings)))
    g.add((s, INST["NumberOfRiser"], Literal(stair.NumberOfRiser)))
    g.add((s, RDFS.label, Literal(stair.name)))

for door in door_list:
    s = INST[str(door.name).replace(" ","").replace(":","_")]
    g.add((s, RDF.type, BEO.Door))
    g.add((s, RDF.type, INST["IfcDoor"]))
    g.add((s, INST["width"], Literal(door.width)))
    g.add((s, INST["height"], Literal(door.height)))
    g.add((s, RDFS.label, Literal(door.name)))
    

# Store the TTL
import os
g.serialize(destination = "house.ttl", format = "turtle")
print("Created house.ttl in folder:")
print(str(os.getcwd()))

Created house.ttl in folder:
/Users/stefan/Repositories/FireBIM/SSolDAC2024/What-the-check/IFC Conversion


In [73]:
# Iterate over triples in store and print them out.
print("--- printing raw triples ---")
for s, p, o in g:
    print((s, p, o))

--- printing raw triples ---
(rdflib.term.URIRef('https://example.org#M_Single-Flush_0915x2032mm_399501'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('M_Single-Flush:0915 x 2032mm:399501'))
(rdflib.term.URIRef('https://example.org#M_Single-Flush_0915x2134mm_358480'), rdflib.term.URIRef('https://example.org#width'), rdflib.term.Literal('914.9999999999999', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://example.org#Door-Exterior-Double_Door-Exterior-Double_391745'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://w3id.org/beo#Door'))
(rdflib.term.URIRef('https://example.org#M_Single-Flush_0915x2134mm_378185'), rdflib.term.URIRef('https://example.org#height'), rdflib.term.Literal('2134.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://example.org#M_Single-Flush_0915x2134mm_401572'), rdfl